In [ ]:
import os
import json

# Get the current working directory
cwd = os.getcwd()

# Navigate to the parent directory
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))

# Path to the directory containing train2014 images
image_dir = os.path.join(parent_dir, "NLP641FinalProject", "COCO_dataset", "train2014")

# Path to the captions mapping JSON file
json_file = os.path.join(parent_dir, "NLP641FinalProject", "COCO_dataset", "captions_mapping_train.json")


In [ ]:
with open(json_file, "r") as f:
    captions_mapping = json.load(f)

image_files = os.listdir(image_dir)

In [ ]:
len(captions_mapping), len(image_files)

In [ ]:
# for image_file in image_files:
#     image_path = 'COCO_dataset/val2014/' + image_file
#     if image_path not in captions_mapping:
#         os.remove(image_path)
#         print(f"Image {image_path} deleted.")

In [ ]:
len(captions_mapping), len(image_files)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0

# Load the base model
base_model = EfficientNetB0(weights='imagenet', include_top=False)
base_model.trainable = False

# Preprocess the image
image_path = 'dog.jpeg'
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
img_tensor = tf.expand_dims(img_array, axis=0)  # Add batch dimension

# Pass the image tensor through the model
cnn_model = tf.keras.Model(inputs=base_model.input, outputs=base_model.output)
output = cnn_model(img_tensor)

print(output.shape)


In [ ]:
# cnn_model.summary()

In [ ]:
import tensorflow as tf

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Set Metal as the default compute device
# tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)

# Load ResNet50 without top layers
resnet50 = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',  # Or specify None if you don't want to use pre-trained weights
    input_shape=(224, 224, 3)  # Adjust input shape according to your needs
)
resnet50.trainable = False  # Freeze the weights of the ResNet50 base model

# Define additional layers
output = resnet50.output
output = tf.keras.layers.Reshape((-1, 2048))(output)  # Reshape the output to match the desired shape
output = tf.keras.layers.Dense(1280)(output)  # Add a dense layer to reduce dimensionality


In [ ]:
# Create the model
cnn_model = tf.keras.Model(inputs=resnet50.input, outputs=output)

# Load and preprocess the image
image_path = 'dog.jpeg'
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.keras.applications.resnet50.preprocess_input(img_array)  # Preprocess input for ResNet50

# Process the image through the model
features = cnn_model.predict(tf.expand_dims(img_array, axis=0))

print(features.shape)  # Check the shape of the extracted features


In [ ]:
# cnn_model.summary()

In [14]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2

# Load the base model
base_model = MobileNetV2(weights='imagenet', include_top=False)
base_model.trainable = False

# Preprocess the image
image_path = 'dog.jpeg'
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
img_tensor = tf.expand_dims(img_array, axis=0)  # Add batch dimension

# Pass the image tensor through the model
cnn_model = tf.keras.Model(inputs=base_model.input, outputs=base_model.output)
output = cnn_model(img_tensor)

print(output.shape)


/var/folders/3_/_3xy56md31d24tz9rgd5r83w0000gn/T/ipykernel_75717/3198675487.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False)


(1, 7, 7, 1280)


In [23]:
import coremltools as ct

saved_model_path = 'saved_model/'
tf.saved_model.save(cnn_model, saved_model_path)

# Convert the TensorFlow SavedModel to CoreML
model = ct.convert(saved_model_path)

model.save('converted_model.mlpackage')

2024-04-24 12:53:42.012707: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-04-24 12:53:42.013009: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-24 12:53:42.013021: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-04-24 12:53:43.125882: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2024-04-24 12:53:43.126189: I tensorflow/core/common_runtime/pluggable_device/p

In [30]:
import coremltools as ct
import numpy as np
import time
from PIL import Image

# Load the CoreML model
model = ct.models.MLModel('converted_model.mlpackage')

# Prepare input data (replace 'input_image.jpg' with your test image)
image = Image.open('cat.jpeg').resize((224, 224))
image = np.array(image) / 127.0 - 1.0
input_data = {'input_image': image}

# Perform inference and measure time
start_time = time.time()
output = model.predict(input_data)
inference_time = time.time() - start_time

# Print inference result
print("Inference result:", output)

# Print inference time
print("Inference time:", inference_time, "seconds")


KeyError: 'Provided key "input_image", in the input dict, does not match any of the model input name(s), which are: inputs'

In [ ]:
cnn_model.summary()

In [ ]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)